# Installing Dependencies

In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
 df=pd.read_csv(r"C:\Users\swast\OneDrive\Desktop\ML Project DATA\Toxic Comment Detection\jigsaw-toxic-comment-classification-challenge\train.csv\train.csv")

In [3]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
df['comment_text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

# Preprocessing

In [5]:
#Tokenization
from tensorflow.keras.layers import TextVectorization

In [6]:
TextVectorization??

In [7]:
x=df['comment_text']
y=df[df.columns[2:]].values

In [9]:
MAX_FEATURES = 200000 # number of words in the vocab

In [15]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [16]:
vectorizer.adapt(x.values)

In [23]:
vectorized_text = vectorizer(x.values)

In [25]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [26]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [28]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 1800), dtype=tf.int64, name=None), TensorSpec(shape=(None, 6), dtype=tf.int64, name=None))>

# DL Model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [32]:
model=Sequential()
model.add(Embedding(MAX_FEATURES+1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))

model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(6, activation='sigmoid'))

In [33]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                      

In [36]:
history = model.fit(train, epochs=1, validation_data=val)

6981/6981 [==============================] - 7527s 1s/step - loss: 0.0639 - val_loss: 0.0450


# Make Predictions

In [49]:
test_in=vectorizer('I am going to kill you and you will die by my hands you racist piece of shit')

In [50]:
model.predict(np.array([test_in]))

1/1 [==============================] - 0s 124ms/step


array([[0.98779166, 0.15196641, 0.86299634, 0.02194306, 0.72805125,
        0.11047161]], dtype=float32)

In [51]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [52]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [53]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 231ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [54]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

# Evaluate Model

In [55]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [56]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [57]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 195ms/step


1/1 [==============================] - 0s 219ms/step


1/1 [==============================] - 0s 206ms/step


1/1 [==============================] - 0s 188ms/step


1/1 [==============================] - 0s 210ms/step


1/1 [==============================] - 0s 196ms/step


KeyboardInterrupt: 

# Representing in a gradio App 

In [59]:
pip install gradio jinja2

  Using cached gradio-3.27.0-py3-none-any.whl (17.3 MB)
                                              0.0/813.6 kB ? eta -:--:--
     -----------------------------         655.4/813.6 kB 14.0 MB/s eta 0:00:01
     ------------------------------------- 813.6/813.6 kB 13.0 MB/s eta 0:00:00
                                              0.0/57.0 kB ? eta -:--:--
     ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
                                              0.0/286.2 kB ? eta -:--:--
     -------------------------------------- 286.2/286.2 kB 8.6 MB/s eta 0:00:00
                                              0.0/75.3 kB ? eta -:--:--
     ---------------------------------------- 75.3/75.3 kB 4.3 MB/s eta 0:00:00
                                              0.0/84.5 kB ? eta -:--:--
     ---------------------------------------- 84.5/84.5 kB 4.6 MB/s eta 0:00:00
       


[notice] A new release of pip is available: 23.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
model.save('toxicity.h5')

In [61]:
import gradio as gr

In [62]:
model = tf.keras.models.load_model('toxicity.h5')

In [63]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [64]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\swast\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\swast\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\swast\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [65]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://c35ba32e7f7eeb8798.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 141ms/step
